In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loadbidaf/bidaftrain.pkl
/kaggle/input/loadbidaf/bidafw2id.pickle
/kaggle/input/loadbidaf/bidafc2id.pickle
/kaggle/input/loadbidaf/bidafglove_tv.npy
/kaggle/input/loadbidaf/bidafvalid.pkl
/kaggle/input/prediction-bidaf/prediction_trainDataset_BIDAF.txt
/kaggle/input/prediction-bidaf/prediction_validDataset_BIDAF.txt
/kaggle/input/new-squaddataset/train-v2.0.json
/kaggle/input/new-squaddataset/dev-v2.0.json
/kaggle/input/ensemble-input/prediction_validDataset_DistilBERT.txt
/kaggle/input/ensemble-input/prediction_trainDataset_BIDAF.txt
/kaggle/input/ensemble-input/prediction_trainDataset_BiLSTM.txt
/kaggle/input/ensemble-input/prediction_validDataset_BIDAF.txt
/kaggle/input/ensemble-input/prediction_trainDataset_DistilBERT.txt
/kaggle/input/ensemble-input/prediction_validDataset_BiLSTM.txt
/kaggle/input/ensemble-input/prediction_validDataset_BERT.txt
/kaggle/input/ensemble-input/prediction_trainDataset_BERT.txt


# **Preprocess**

In [2]:
def load_json(path):
    '''
    Loads the JSON file of the Squad dataset.
    Returns the json object of the dataset.
    '''
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    print("Length of data: ", len(data['data']))
    print("Data Keys: ", data['data'][0].keys())
    print("Title: ", data['data'][0]['title'])
    
    return data


def parse_data(data:dict)->list:
    '''
    Parses the JSON file of Squad dataset by looping through the
    keys and values and returns a list of dictionaries with
    context, query and label triplets being the keys of each dict.
    '''
    data = data['data']
    qa_list = []

    for paragraphs in data:

        for para in paragraphs['paragraphs']:
            context = para['context']

            for qa in para['qas']:
                
                id = qa['id']
                question = qa['question']
                
                for ans in qa['answers']:
                    answer = ans['text']
                    ans_start = ans['answer_start']
                    ans_end = ans_start + len(answer)
                    
                    qa_dict = {}
                    qa_dict['id'] = id
                    qa_dict['context'] = context
                    qa_dict['question'] = question
                    qa_dict['label'] = [ans_start, ans_end]

                    qa_dict['answer'] = answer
                    qa_list.append(qa_dict)    

    
    return qa_list

In [3]:
import json

train_data = load_json('/kaggle/input/new-squaddataset/train-v2.0.json')
valid_data = load_json('/kaggle/input/new-squaddataset/dev-v2.0.json')

# parse the json structure to return the data as a list of dictionaries

train_list = parse_data(train_data)
valid_list = parse_data(valid_data)
print('--------------------------')

print('Train list len: ',len(train_list))
print('Valid list len: ',len(valid_list))

# converting the lists into dataframes

train_ds = pd.DataFrame(train_list)
val_ds = pd.DataFrame(valid_list)
# train_ds.columns = ['Paragraph', 'Question', 'Answer']
# val_ds.columns = ['Paragraph', 'Question', 'Answer']

Length of data:  442
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Beyoncé
Length of data:  35
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Normans
--------------------------
Train list len:  86821
Valid list len:  20302


In [4]:
train_ds.head()

,id,context,question,label,answer
0,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"[269, 286]",in the late 1990s
1,56be85543aeaaa14008c9065,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"[207, 226]",singing and dancing
2,56be85543aeaaa14008c9066,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"[526, 530]",2003
3,56bf6b0f3aeaaa14008c9601,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"[166, 180]","Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,"[276, 286]",late 1990s


In [5]:
val_ds.head()

,id,context,question,label,answer
0,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"[159, 165]",France
1,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"[159, 165]",France
2,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"[159, 165]",France
3,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"[159, 165]",France
4,56ddde6b9a695914005b9629,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,"[94, 117]",10th and 11th centuries


# **Reading the predictions**

In [6]:
import json
import string
import re
from collections import Counter

filename = '/kaggle/input/ensemble-input/prediction_trainDataset_BIDAF.txt'
with open(filename, 'r', encoding='utf-8') as file:
    prediction_trainDataset_BIDAF = json.load(file)
    
filename = '/kaggle/input/ensemble-input/prediction_validDataset_BIDAF.txt'
with open(filename, 'r', encoding='utf-8') as file:
    prediction_validDataset_BIDAF = json.load(file)

    
filename = '/kaggle/input/ensemble-input/prediction_trainDataset_BiLSTM.txt'
with open(filename, 'r', encoding='utf-8') as file:
    prediction_trainDataset_BiLSTM = json.load(file)
    
filename = '/kaggle/input/ensemble-input/prediction_validDataset_BiLSTM.txt'
with open(filename, 'r', encoding='utf-8') as file:
    prediction_validDataset_BiLSTM = json.load(file)
    
    
filename = '/kaggle/input/ensemble-input/prediction_trainDataset_DistilBERT.txt'
with open(filename, 'r', encoding='utf-8') as file:
    prediction_trainDataset_DistilBERT = json.load(file)
    
filename = '/kaggle/input/ensemble-input/prediction_validDataset_DistilBERT.txt'
with open(filename, 'r', encoding='utf-8') as file:
    prediction_validDataset_DistilBERT = json.load(file)
    
    
filename = '/kaggle/input/ensemble-input/prediction_trainDataset_BERT.txt'
with open(filename, 'r', encoding='utf-8') as file:
    prediction_trainDataset_BERT = json.load(file)
    
filename = '/kaggle/input/ensemble-input/prediction_validDataset_BERT.txt'
with open(filename, 'r', encoding='utf-8') as file:
    prediction_validDataset_BERT = json.load(file)

# **Conbining above data into single train and valid**

In [7]:
val_ds = val_ds.drop('label', axis=1)
train_ds = train_ds.drop('label', axis=1)

In [8]:
val_ds

,id,context,question,answer
0,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France
1,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France
2,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France
3,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France
4,56ddde6b9a695914005b9629,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,10th and 11th centuries
...,...,...,...,...
20297,5737aafd1c456719005744ff,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,sthène
20298,5737aafd1c456719005744ff,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,sthène
20299,5737aafd1c456719005744ff,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,sthène
20300,5737aafd1c456719005744ff,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,sthène


In [9]:
ct1= 0
ct2= 0
for index, row in val_ds.iterrows():
    id = row['id']
    context = row['context']
    question = row['question']
    if id in prediction_validDataset_BIDAF:
        val_ds.at[index, 'BIDAF_ans'] = prediction_validDataset_BIDAF[id][0]
        val_ds.at[index, 'BIDAF_f1'] = prediction_validDataset_BIDAF[id][1]
    else:
        ct1 += 1
        val_ds.at[index, 'BIDAF_ans'] = 0
        val_ds.at[index, 'BIDAF_f1'] = 0
    if id in prediction_validDataset_BiLSTM:
        val_ds.at[index, 'BiLSTM_ans'] = prediction_validDataset_BiLSTM[id][0]
        val_ds.at[index, 'BiLSTM_f1'] = prediction_validDataset_BiLSTM[id][1]
    else:
        ct2 += 1
        val_ds.at[index, 'BiLSTM_ans'] = 0
        val_ds.at[index, 'BiLSTM_f1'] = 0
    if id in prediction_validDataset_BERT:
        val_ds.at[index, 'BERTCased_ans'] = prediction_validDataset_BERT[id][0]
        val_ds.at[index, 'BERTCased_f1'] = prediction_validDataset_BERT[id][1]
    else:
        ct1 += 1
        val_ds.at[index, 'BERTCased_ans'] = 0
        val_ds.at[index, 'BERTCased_f1'] = 0
    val_ds.at[index, 'DistilBERT_ans'] = prediction_validDataset_DistilBERT[id][0]
    val_ds.at[index, 'DistilBERT_f1'] = prediction_validDataset_DistilBERT[id][1]

In [10]:
print(ct1)
print(ct2)

2502
0


In [13]:
train_ds['BERTCased_ans'] = train_ds['BERTCased_ans'].astype(object)

In [14]:
for index, row in train_ds.iterrows():
    id = row['id']
    context = row['context']
    question = row['question']
    if id in prediction_trainDataset_BIDAF:
        train_ds.at[index, 'BIDAF_ans'] = prediction_trainDataset_BIDAF[id][0]
        train_ds.at[index, 'BIDAF_f1'] = prediction_trainDataset_BIDAF[id][1]
    else:
        train_ds.at[index, 'BIDAF_ans'] = 0
        train_ds.at[index, 'BIDAF_f1'] = 0
    if id in prediction_trainDataset_BiLSTM:
        train_ds.at[index, 'BiLSTM_ans'] = prediction_trainDataset_BiLSTM[id][0]
        train_ds.at[index, 'BiLSTM_f1'] = prediction_trainDataset_BiLSTM[id][1]
    else:
        train_ds.at[index, 'BiLSTM_ans'] = 0
        train_ds.at[index, 'BiLSTM_f1'] = 0
    if id in prediction_trainDataset_BERT:
        train_ds.at[index, 'BERTCased_ans'] = prediction_trainDataset_BERT[id][0]
        train_ds.at[index, 'BERTCased_f1'] = prediction_trainDataset_BERT[id][1]
    else:
        train_ds.at[index, 'BERTCased_ans'] = 0
        train_ds.at[index, 'BERTCased_f1'] = 0
    if id in prediction_trainDataset_DistilBERT:
        train_ds.at[index, 'DistilBERT_ans'] = prediction_trainDataset_DistilBERT[id][0]
        train_ds.at[index, 'DistilBERT_f1'] = prediction_trainDataset_DistilBERT[id][1]
    else:
        train_ds.at[index, 'DistilBERT_ans'] = 0
        train_ds.at[index, 'DistilBERT_f1'] = 0

In [ ]:
# prediction_trainDataset_BERT.keys()

In [15]:
train_ds

,id,context,question,answer,BIDAF_ans,BIDAF_f1,BiLSTM_ans,BiLSTM_f1,BERTCased_ans,BERTCased_f1,DistilBERT_ans,DistilBERT_f1
0,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,2003,0.000000,l,0.153846,0,0.000000,late 1990s,0.758621
1,56be85543aeaaa14008c9065,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,"singing and dancing competitions as a child , ...",0.146341,(/,0.181818,various singing and dancing,0.857143,singing and dancing,1.000000
2,56be85543aeaaa14008c9066,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,late 1990s,0.000000,l,0.000000,late 1990s,0.000000,2003),1.000000
3,56bf6b0f3aeaaa14008c9601,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",houston,0.666667,(/,0.133333,Houston,0.000000,"Houston,",0.695652
4,56bf6b0f3aeaaa14008c9602,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,2003,0.000000,l,0.200000,1990s,0.666667,1990s,0.705882
...,...,...,...,...,...,...,...,...,...,...,...,...
86816,5735d259012e2f140011a09d,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,Oregon,oregon,1.000000,n o,0.444444,"Oregon, United States. This activity has been ...",0.000000,"Oregon,",1.000000
86817,5735d259012e2f140011a09e,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Rangoon,rangoon,1.000000,n o,0.500000,Rangoon) of Myanmar,0.000000,Rangoon),1.000000
86818,5735d259012e2f140011a09f,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Minsk,xi'an,0.000000,"Kathmandu Metropolitan City (KMC), in order to...",0.230769,"Eugene, Oregon",0.000000,Minsk,1.000000
86819,5735d259012e2f140011a0a0,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,1975,1975,1.000000,C,0.000000,0,0.000000,1975,1.000000


# **Processing them**

In [16]:
import pickle

word2idx = {}

with open('/kaggle/input/loadbidaf/bidafw2id.pickle','rb') as handle:
    word2idx = pickle.load(handle)

In [17]:
len(word2idx)

96776

# **Processing Data**

In [18]:
num_rows = len(val_ds)
valid_rows_BIDAF = val_ds[val_ds['BIDAF_ans'] != 0]
valid_rows_BiLSTM = val_ds[val_ds['BiLSTM_ans'] != 0] 
valid_rows_DistilBERT = val_ds[val_ds['DistilBERT_ans'] != 0] 
valid_rows_BERT = val_ds[val_ds['BERTCased_ans'] != 0] 
f1_validDataset_BIDAF = valid_rows_BIDAF['BIDAF_f1'].sum() / len(valid_rows_BIDAF)
f1_validDataset_BiLSTM = valid_rows_BiLSTM['BiLSTM_f1'].sum() / len(valid_rows_BiLSTM)
f1_validDataset_DistilBERT = valid_rows_DistilBERT['DistilBERT_f1'].sum() / len(valid_rows_DistilBERT)
f1_validDataset_BERT = valid_rows_BERT['BERTCased_f1'].sum() / len(valid_rows_BERT)

em_validDataset_BIDAF = len(valid_rows_BIDAF[valid_rows_BIDAF['BIDAF_f1'] == 1]) / len(valid_rows_BIDAF)
em_validDataset_BiLSTM = len(valid_rows_BiLSTM[valid_rows_BiLSTM['BiLSTM_f1'] == 1]) / len(valid_rows_BiLSTM)
em_validDataset_DistilBERT = len(valid_rows_DistilBERT[valid_rows_DistilBERT['DistilBERT_f1'] == 1]) / len(valid_rows_DistilBERT)
em_validDataset_BERT = len(valid_rows_BERT[valid_rows_BERT['BERTCased_f1'] == 1]) / len(valid_rows_BERT)

# Print the average values
print("Average BIDAF_f1:", f1_validDataset_BIDAF)
print("Average BiLSTM_f1:", f1_validDataset_BiLSTM)
print("Average DistilBERT_f1:", f1_validDataset_DistilBERT)
print("Average BERT_f1:", f1_validDataset_BERT)

Average BIDAF_f1: 0.6680005949338813
Average BiLSTM_f1: 0.28518230394810734
Average DistilBERT_f1: 0.7530296296892632
Average BERT_f1: 0.4025203999249132


In [19]:
num_rows = len(train_ds)
train_rows_BIDAF = train_ds[train_ds['BIDAF_ans'] != 0]
train_rows_BiLSTM = train_ds[train_ds['BiLSTM_ans'] != 0] 
train_rows_DistilBERT = train_ds[train_ds['DistilBERT_ans'] != 0] 
train_rows_BERT = train_ds[train_ds['BERTCased_ans'] != 0] 
f1_trainDataset_BIDAF = train_rows_BIDAF['BIDAF_f1'].sum() / len(train_rows_BIDAF)
f1_trainDataset_BiLSTM = train_rows_BiLSTM['BiLSTM_f1'].sum() / len(train_rows_BiLSTM)
f1_trainDataset_DistilBERT = train_rows_DistilBERT['DistilBERT_f1'].sum() / len(train_rows_DistilBERT)
f1_trainDataset_BERT = train_rows_BERT['BERTCased_f1'].sum() / len(train_rows_BERT)

em_trainDataset_BIDAF = len(train_rows_BIDAF[train_rows_BIDAF['BIDAF_f1'] == 1]) / len(train_rows_BIDAF)
em_trainDataset_BiLSTM = len(train_rows_BiLSTM[train_rows_BiLSTM['BiLSTM_f1'] == 1]) / len(train_rows_BiLSTM)
em_trainDataset_DistilBERT = len(train_rows_DistilBERT[train_rows_DistilBERT['DistilBERT_f1'] == 1]) / len(train_rows_DistilBERT)
em_trainDataset_BERT = len(train_rows_BERT[train_rows_BERT['BERTCased_f1'] == 1]) / len(train_rows_BERT)

# Print the average values
print("Average BIDAF_f1:", f1_trainDataset_BIDAF)
print("Average BiLSTM_f1:", f1_trainDataset_BiLSTM)
print("Average DistilBERT_f1:", f1_trainDataset_DistilBERT)
print("Average BERT_f1:", f1_trainDataset_BERT)

Average BIDAF_f1: 0.6512755191469993
Average BiLSTM_f1: 0.26962483004132115
Average DistilBERT_f1: 0.8781218393573155
Average BERT_f1: 0.40688574065643784


# **Individual Model Result**

In [20]:
print("=============================================================")
print("EM Score on Validation Dataset using BIDAF Model = ",end="")
print(em_validDataset_BIDAF)
print("F1 Score on Validation Dataset using BIDAF Model = ",end="")
print(f1_validDataset_BIDAF)
print("---------------------------------") 
print("EM Score on Train Dataset using BIDAF Model = ",end="")
print(em_trainDataset_BIDAF)
print("F1 Score on Train Dataset using BIDAF Model = ",end="")
print(f1_trainDataset_BIDAF)
print("=============================================================")
print("EM Score on Validation Dataset using BiLSTM Model = ",end="")
print(em_validDataset_BiLSTM)
print("F1 Score on Validation Dataset using BiLSTM Model = ",end="")
print(f1_validDataset_BiLSTM)
print("---------------------------------") 
print("EM Score on Train Dataset using BiLSTM Model = ",end="")
print(em_trainDataset_BiLSTM)
print("F1 Score on Train Dataset using BiLSTM Model = ",end="")
print(f1_trainDataset_BiLSTM)
print("=============================================================")
print("EM Score on Validation Dataset using DistilBERT Model = ",end="")
print(em_validDataset_DistilBERT)
print("F1 Score on Validation Dataset using DistilBERT Model = ",end="")
print(f1_validDataset_DistilBERT)
print("---------------------------------") 
print("EM Score on Train Dataset using DistilBERT Model = ",end="")
print(em_trainDataset_DistilBERT)
print("F1 Score on Train Dataset using DistilBERT Model = ",end="")
print(f1_trainDataset_DistilBERT)
print("=============================================================")
print("EM Score on Validation Dataset using BERT Model = ",end="")
print(em_validDataset_BERT)
print("F1 Score on Validation Dataset using BERT Model = ",end="")
print(f1_validDataset_BERT)
print("---------------------------------") 
print("EM Score on Train Dataset using BERT Model = ",end="")
print(em_trainDataset_BERT)
print("F1 Score on Train Dataset using BERT Model = ",end="")
print(f1_trainDataset_BERT)
print("=============================================================")

EM Score on Validation Dataset using BIDAF Model = 0.5447480447480447
F1 Score on Validation Dataset using BIDAF Model = 0.6680005949338813
---------------------------------
EM Score on Train Dataset using BIDAF Model = 0.49979049303971324
F1 Score on Train Dataset using BIDAF Model = 0.6512755191469993
EM Score on Validation Dataset using BiLSTM Model = 0.0004433060782188947
F1 Score on Validation Dataset using BiLSTM Model = 0.28518230394810734
---------------------------------
EM Score on Train Dataset using BiLSTM Model = 0.0005989334377627533
F1 Score on Train Dataset using BiLSTM Model = 0.26962483004132115
EM Score on Validation Dataset using DistilBERT Model = 0.5613732637178603
F1 Score on Validation Dataset using DistilBERT Model = 0.7530296296892632
---------------------------------
EM Score on Train Dataset using DistilBERT Model = 0.705635733290333
F1 Score on Train Dataset using DistilBERT Model = 0.8781218393573155
EM Score on Validation Dataset using BERT Model = 0.1339

# **Emsembling Model**

In [22]:
ct_train = 0
final_f1_train = 0
final_em_train = 0
for index, row in train_ds.iterrows():
    local_f1 = 0
    local_f1 = max(local_f1, row.BIDAF_f1)
    local_f1 = max(local_f1, row.BiLSTM_f1)
    local_f1 = max(local_f1, row.BERTCased_f1)
#     local_f1 = max(local_f1, row.DistilBERT_f1)
    if(local_f1 == 0):
        continue
    if(local_f1 == 1):
        final_em_train += 1
    final_f1_train += local_f1
    ct_train += 1


In [23]:
ct_dev = 0
final_f1_dev = 0
final_em_dev = 0
for index, row in val_ds.iterrows():
    local_f1 = 0
    local_f1 = max(local_f1, row.BIDAF_f1)
    local_f1 = max(local_f1, row.BiLSTM_f1)
    local_f1 = max(local_f1, row.BERTCased_f1)
#     local_f1 = max(local_f1, row.DistilBERT_f1)
    if(local_f1 == 0):
        continue
    if(local_f1 == 1):
        final_em_dev += 1
    final_f1_dev += local_f1
    ct_dev += 1


In [24]:
print("=============================================================")
print("Ensembled F1 Score on Train Dataset:  ",end='')
print(final_f1_train/ct_train)
print("Ensembled EM Score on Train Dataset:  ",end='')
print(final_em_train/ct_train)
print("-------------------------------------------------")
print("Ensembled F1 Score on Dev Dataset:  ",end='')
print(final_f1_dev/ct_dev)
print("Ensembled EM Score on Dev Dataset:  ",end='')
print(final_em_dev/ct_dev)
print("=============================================================")

Ensembled F1 Score on Train Dataset:  0.8157256536261616
Ensembled EM Score on Train Dataset:  0.5692873704632565
-------------------------------------------------
Ensembled F1 Score on Dev Dataset:  0.8305768173069266
Ensembled EM Score on Dev Dataset:  0.616244939271255
